In [2]:
import numpy as np
import pandas as pd
import datetime
from datetime import datetime, timedelta

In [3]:
df_price = pd.read_csv('./data/prices_clean.csv')
df_sales = pd.read_csv('./data/sales_clean.csv')

In [4]:
print(df_price.shape)
df_price.head(1)

(1536776, 5)


,Unnamed: 0,item,store_code,date,sell_price
0,0,ACCESORIES_1_001,BOS_1,2013-07-31,11.753875


In [5]:
print(df_sales.shape)
df_sales.head(1)

(1951360, 4)


,Unnamed: 0,id,date,sales
0,0,ACCESORIES_1_001_BOS_1,2011-01,0
